In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Repro + imports
import os, random, numpy as np, tensorflow as tf
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

# Paths (point to dataset_200)
BASE = '/content/drive/MyDrive/Capstone2025_DeepfakeDetection'
DATA = f'{BASE}/data/dataset_200/frames_cropped_split'   # <-- your new dataset
MODEL_DIR = f'{BASE}/models_fast200'
os.makedirs(MODEL_DIR, exist_ok=True)

IMG_SIZE   = (224, 224)
BATCH_SIZE = 32
EPOCHS     = 15


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Simple rescale-only (same as before)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    os.path.join(DATA, 'train'),
    target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=True, seed=SEED
)
val_generator = datagen.flow_from_directory(
    os.path.join(DATA, 'val'),
    target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=False
)
test_generator = datagen.flow_from_directory(
    os.path.join(DATA, 'test'),
    target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='binary', shuffle=False
)

# Optional: class weights (keeps recall healthier if slightly imbalanced)
from collections import Counter
counts = Counter(train_generator.classes)  # {0: real, 1: fake}
total = sum(counts.values())
class_weight = {0: total/(2.0*counts[0]), 1: total/(2.0*counts[1])}
class_weight


Found 5600 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


{0: 1.0, 1: 1.0}

In [ ]:
def add_head(base, dropout=0.3):
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    return Model(base.input, out)

def build_mobilenetv2(input_shape=(224,224,3)):
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base.trainable = False   # freeze for a fast first stage (CPU-friendly)
    model = add_head(base, dropout=0.3)
    model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def get_callbacks(name):
    return [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6, verbose=1),
        ModelCheckpoint(os.path.join(MODEL_DIR, f'{name}_best.keras'),
                        monitor='val_accuracy', save_best_only=True, verbose=1)
    ]


In [ ]:
mnv2 = build_mobilenetv2()

history = mnv2.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('mobilenetv2_fast'),
    class_weight=class_weight,
    verbose=1
)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.7124 - loss: 0.6403 
Epoch 1: val_accuracy improved from -inf to 0.83500, saving model to /content/drive/MyDrive/Capstone2025_DeepfakeDetection/models_fast200/mobilenetv2_fast_best.keras
175/175 ━━━━━━━━━━━━━━━━━━━━ 2678s 15s/step - accuracy: 0.7127 - loss: 0.6397 - val_accuracy: 0.8350 - val_loss: 0.3656 - learning_rate: 0.0010
Epoch 2/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 816ms/step - accuracy: 0.8624 - loss: 0.2969
Epoch 2: val_accuracy improved from 0.83500 to 0.88583, saving model to /content/drive/MyDrive/Capstone2025_DeepfakeDetection/models_fast200/mobilenetv2_fast_best.keras
175/175 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8625 - loss: 0.2968 - val_accuracy: 0.8858 - val_loss: 0.2551 - learning_rate: 0.0010
Epoch 3/15
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 809ms/step - accuracy: 0.9026 - loss: 0.2183
Epoch 3: val_accuracy improved from 0.88583 to 0.89750, saving model to /content/drive/MyDrive/Capstone2025_Deepfake

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np, json, time

# Load the best checkpoint if saved
import tensorflow as tf, os
best_path = os.path.join(MODEL_DIR, 'mobilenetv2_ft_best.keras')
if not os.path.exists(best_path):
    best_path = os.path.join(MODEL_DIR, 'mobilenetv2_fast_best.keras')
mnv2_best = tf.keras.models.load_model(best_path)

# Evaluate
test_probs = mnv2_best.predict(test_generator, verbose=1)
test_pred  = (test_probs >= 0.5).astype(int).ravel()  # keep 0.5 default; easy to change later
y_true     = test_generator.classes

print("Accuracy:", (test_pred == y_true).mean())
print(classification_report(y_true, test_pred, target_names=['Real','Fake']))
print("Confusion Matrix:\n", confusion_matrix(y_true, test_pred))

# Save a single-file model for deployment
single_path = os.path.join(MODEL_DIR, 'mobilenetv2_final.keras')
mnv2_best.save(single_path)
print("Saved model →", single_path)


38/38 ━━━━━━━━━━━━━━━━━━━━ 773s 21s/step
Accuracy: 0.9041666666666667
              precision    recall  f1-score   support

        Real       0.89      0.92      0.91       600
        Fake       0.92      0.89      0.90       600

    accuracy                           0.90      1200
   macro avg       0.90      0.90      0.90      1200
weighted avg       0.90      0.90      0.90      1200

Confusion Matrix:
 [[551  49]
 [ 66 534]]
Saved model → /content/drive/MyDrive/Capstone2025_DeepfakeDetection/models_fast200/mobilenetv2_final.keras
